In [ ]:
# Data type constraints
Datatype              Example                                Python data type
Text data             First name, last name, address, ...    str
Integers              # Subscribers, # products sold, ...    int
Decimals              Temperature, $ exchange rates, ...     float
Binary                Is married, new customer, yes/no, ...  bool
Dates                 Order dates, ship dates, ...           datetime
Categories            Marriage status, gender ...            category

# Strings to integers
# Import CSV file and output header
sales = pd.read_csv('sales.csv')
sales.head()
    #prints below output
    SalesOrderID    Revenue    Quantity
0   43659           23153$     12
1   43660           1457$      2

# Set data types of columns
sales.dtypes
    #prints below output
SalesOrderID    int64
Revenue         object
Quantity        int64
dtype: object

# Get dataframe information
sales.info()
    #prints below output
<class 'pandas.core.frame.DataFrame'> 
RangeIndex: 31465 entries, 0 to 31464 
Data columns (total 3 columns):
SalesOrderID    31465 non-null int64
Revenue         31465 non-null object
Quantity        31465 non-null int64
dtypes: int64(2), object(1)
memory usage: 737.5+ KB

# print sum of all Revenue column 
sales['Revenue'].sum()  # prints '23153$1457$36865$32474$472$27510$16158$5694$6876$40487$807$6893$9153$6895$4216..

# Remove $ from Revenue column
sales['Revenue'] = sales['Revenue'].str.strip('$')
sales['REvenue'] = sales['Revenue'].astype('int')

# Verify that Revenue is now an integer
assert sales['Revenue'].dtype == 'int' #prints nothing if the condition is met

# This will not pass
assert 1+1 == 3 
#prints
AssertionError                   Traceback (most recent call last)
        assert 1+1 == 3
AssertionError:
    

# Numeric or categorical?
marriage_status(column) - values(3, 1, 2)
0 = Never married, 1 = Married, 2 = Separated, 3 Divorced

df['marriage_status'].describe() #prints (marriage_status - mean 1.4, std 0.20, min 0.00, 50% 1.8 ...)

#convert to catogorical
df['marriage_status'] = df['marriage_status'].astype('category') 
df['marriage_status'].describe() #prints (marriage_status - count 241, unique 4, top 1, freq 120)

In [ ]:
# Data range constraints
How to deal with out of range data?
• Dropping data
• Setting custom minimums and maximums
• Treat as missing and impute
• Setting custom value depending on business assumptions

# Movie example (when avg_rating should be 1-5 but has values out of range >5 in dataframe)
import pandas as pd

# output Movies with rating > 5
movies[movies['avg_ratiing'] > 5]

# Drop values using filtering
movies = movies[movies['avg_rating'] <= 5]

# Drop values using .drop()
movies.drop(movies[movies['avg_rating'] > 5].index, inplace=True)

# Assert the results
assert movies['avg_rating'].max() <= 5

# Convert avg_rating > 5 to 5
movies.loc[movies['avg_rating'] > 5, 'avg_rating'] = 5


# Subscriptions example(when subcription dates have future dates which should be < dt.date.today())
import datetime as dt
import pandas as pd
# Convert to date type
user_signups['subscription_date'] = pd.to_datetime(user_signups['subscription_date']).dt.date

today_date = dt.date.today()

# Drop the data
# Drop values using filtering
user_signups = user_signups[user_signups['subscription_date'] < today_date]
# Drop values using .drop()
user_signups.drop(user_signups[user_signups['subscription_date'] < today_date].index, inplace=True)

# Hardcode dates with upper limit
# Drop values using filtering
user_signups.loc[user_signups['subscription_date'] < today_date, 'subscription_date'] = today_date
#Assert is True
assert user_signups.subscription_date.max().date() <= today_date

In [ ]:
# Uniqueness constraints
What are duplicate values?
Most columns have the same values
first_name last_name  address                                    height weight
Justin     Saddlemyer Boulevard du Jardin Botanique 3, Bruxelles 193 cm 87 kg
Justin     Saddlemyer Boulevard du Jardin Botanique 3, Bruxelles 194 cm 87 kg #only diff in height column

# Why do they happen
Data entry & Human error
Join or merge errors
bugs and design errors

How to find duplicate rows?
The .duplicated() method
subset : List of column names to check for duplication.
keep: Whether to keep first ('first'), last ('Last') or all (False) duplicate values.
    
# Get duplicate roes across all columns
duplicates = height_weight.duplicated()
print(duplicates) #Returns True for duplicated values and False for non duplicated values

# column names to check for duplication
column_names = ['first_name', 'last_name', 'address']
duplicates = height_weight.duplicated(subset = column_names, keep = False)

# output duplicate values
height_weight[duplicated].sort_values(by = 'first_name') #returns all complete and incomplete duplicates

How to treat duplicate values?
The .drop_duplicates () method
subset : List of column names to check for duplication.
keep: Whether to keep first ('first'), last ('last') or all (False) duplicate values.
inplace: Drop duplicated rows directly inside DataFrame without creating new object (True).
    
# Drop duplicates
height_weight.drop_duplicates(inplace = True) # dropes complete duplicates

# how to treat incomplete duplicates #use .groupby() and .agg() methods
# Group by column names and produce statistical summaries
column_names = ['first_name', 'last_name', 'address']
summaries = {'height': 'max', 'weight':'mean'} # takes max value in height and mean of weights among each duplicate
height_weight = height_weight.groupby(by = column_names).agg(summaries).reset_index()

# Make sure aggregation is done
duplicates = height_weight.duplicated(subset = column_names, keep=False)
height_weight[duplicates].sort_values(by = 'first_name') # returns empty if duplicates are cleared

In [ ]:
# Text and categorical data problems
# Categories and membership constraints
# Predefined finite set of categories
Type of data                Example values           Numeric representation
Marriage Status             unmarried, married       0,1                     #Marriage status can only be unmarried _or_ married
Household Income Category   0-20K, 20-40K, ...       0,1,..
Loan Status                 default, payed, no_Loan  0,1,2

# Some of the datasets can have out of range values

# How do we treat these problems?
Dropping Data
Remapping Categories
Inferring Categories

# Finding inconsistent categories
inconsistent_categories = set(study_data['blood_type']).difference(categories['blood_type'])
print(inconsistent_categories) # prints out of range values such as z+ in blood types

# Get and print rows with inconsistent categories
inconsistent_rows = study_data['blood_type'].isin(inconsistent_categories)
study_data[inconsistent_rows] # prints complete rows with out of range values

# Dropping inconsistent categories
inconsistent_data = study_data[inconsistent_rows]
consistent_data = study_data[~inconsistent_rows]


# What type of errors could we have?
I) Value inconsistency
    • Inconsistent fields: 'married', 'Maried', 'UNMARRIED' 'not married' ..
    • Trailing white spaces: _ 'married ' 'married
II) Collapsing too many categories to few
    • Creating new groups: 0-20K, 20-40K categories ... from continuous household income data
    • Mapping groups to new ones: Mapping household income categories to 2 'rich', 'poor' 
III) Making sure data is of type category (seen in Chapter 1)

# Value consistency
Capitalization: 'married', 'Married', 'UNMARRIED' 'unmarried'
# capitalize
marriage_status['marriage_status'] = marriage_status['marriage_status'].str.upper()  #all values are turned in uppercase
# lowercase
marriage_status['marriage_status'] = marriage_status['marriage_status'].str.lower()  #all values are turned in lowercase

Trailing spaces: 'married ', 'married', 'unmarried',' Unmarried',
# strip all spaces
demographics = demographics['marriage_status'].str.strip() # removes white spaces in beggining and end


# collapsing data into categories
# Using cut() - create category ranges and names
ranges = [0, 200000, 500000, np.inf]
group_names = ['0-200k', '200k-500k', '500k+']
# Create income group column
demographics['income_group'] = pd.cut(demographocs['household_income'], bins=ranges, labels=group_names)
demographics[['income_group', 'household_income']] # outputs 0-200k for 188923 and 500k+ for 778533 in a category column

# Collapsing data into categories
Map categories to fewer ones: reducing categories in categorical column.
operating system column is: 'Microsoft', 'MacOS', 'IOS', 'Android', 'Linux'
operating system column should become: 'DesktopOS', 'MobileOS'
    
# creating mapping dictionary and replace
mapping = {'Microsoft':'DesktopOS', 'MacOS':'DesktopOS', 'Linux':'DesktopOS', 'IOS':'MobileOS', 'Android':'MobileOS'}

devices['operating_system'] = devices['operating_system'].replace(mapping)
devices['operating_system'].unique() # prints array(['DesktopOS', 'MobileOS'], dtype=object)

# Common text data problems
1) Data inconsistency:
+96171679912 or 0096171679912 or ..?
2) Fixed length violations:
Passwords needs to be at least 8 characters
3) Typos:
+961.71.679912

Name                Phone_number
Gil B. Silva        001-195-492-2338
Prescott D. Hardin   +1-297-996-4904    <-- Inconsistent data format
Benedict G. Valdez  001-969-820-3536
Reece M. Andrews                4138    <-- Length violation

# Fixing the phone number column
phones['Phone_number'] = phones['phone_number'].str.replace('+','00') #replace + with 00

# Replace phone numbers with lower than 10 digits to NaN
digits = phones['Phone_number'].str.len()
phones.loc[digits < 10, 'Phone_number'] = np.nan
phones # prints NaN where digits are < 10

# Assert min phone number length is 10
assert digits.min() >= 10

# Assert all numbers do not have "+" or "-"
assert phones['Phone_number'].str.contains('+|-').any() == False

# Replace letters with nothing
phones['Phone_number'] = phones['Phone_number'].str.replace(r'\D+', '') # replaces anything that is not digit with nothing


In [ ]:
# Advance data problems
# Uniformity
Column         Unit
Temperature    32°C is also 89.6°F
Weight         70 Kg is also 11 st.
Date           26-11-2019 is also 26, November, 2019
Money          100$ is also 10763.90¥

Date        Temperature
06.03.19    16.0
07.03.19    62.6  <-- #this can be fahrenheit (can be known by plotting)

c = (F - 32) * 5/9 #Formula to convert fahrenheit to celsius

temp_fah = temperatures.loc[temperatures['Temperature'] > 40, 'Temperature']
temp_cels = (temp_fah - 32) * 5/9
temperatures.loc[temperatures['Temperatures'] > 40, 'Temperatures'] = temp_cels

# Assert conversion is correct
assert temperatures['Temperature'].max() < 40


# Treating date data
Birthday          First name   Last name
27/27/19          Rowan        Nunez       ??
03-29-19          Brynn        Yang        MM-DD-YY
March 3rd, 2019   Sophia       Reilly      Month D, YYYY

# Datetime formatting
datetime is useful for representing dates
Date                datetime format
25-12-2019          %d-%m-%Y
December 25th 2019  %c
12-25-2019          %m-%d-%Y

• Can recognize most formats automatically
• Sometimes fails with erroneous or
• unrecognizable formats
pandas.to_datetime()

birthdays['Birthday'] = pd.to_datetime(birrthdays['Birthday'], infer_datetime_format = True, errors='coerce') #Attempt to infer format from each date, return NA for rows where conversion failed
birthdays['Birthday'] = birthdays['Birthday'].dt.strftime('%d-%m-%y')

Treating ambiguous date data
Is 2019-03-08 in August or March?
• Convert to NA and treat accordingly
• Infer format by understanding data source
• Infer format by understanding previous and subsequent data in DataFrame

# Cross field validation
flight_number     economy_class    business_class    first_class    total_passengers
DL140             100           +  60             +  40           = 200
BA248             130           +  100            +  70           = 300  # verify if economy_class+business_class+first_class = total_passengers

# The use of multiple fields in a dataset to sanity check data integrity
sum_classes = flights[['economy_class','business_class','first_class']].sum(axis=1)
passenger_equ = sum_classes == flights['total_passengers']

# find and filter out rows with inconsistent passenger totals
inconsistent_pass = flights[~passenger_equ]
consistent_pass = flights[passenger_equ]

# ex 2
user_id Age Birthday
32985   22  1998-03-02
94387   27  1993-12-04
34236   42  1978-11-24
12551   31  1989-01-03
55212   18  2002-07-02     #verify if current_year - birthyear = Age
 
import pandas as pd 
import datetime as dt

# Convert to datetime and get today's date
users['Birthday'] = pd.to_datetime(users['Birthday'])
today = dt.date.today()
# For each row in the Birthday column, calculate year difference age_manual = today.year - users['Birthday'].dt.year
# Convert to datetime and get today's date
users['Birthday'] = pd.to_datetime(users['Birthday'])
today = dt.date.today()
# For each row in the Birthday column, calculate year difference
age_manual = today.year - users['Birthday'].dt.year
# Find instances where ages match
age_equ = age_manual == users['Age']
# Find and filter out rows with inconsistent age
inconsistent_age = users [~age_equ]
consistent_age = users [age_equ]

# What to do when we catch inconsistencies
Dropping Data
Set to missing and impute
Apply rules from domain knowledge


# What is missing data  (can be represented as NA, nan, 0, ., ...), Occurs when no data value is stored for a variable in an observation, Caused by technical error or human error
# Return missing values 
airquality.isna()
     Date   Temperature  C02
2119 False  False        False
2451 False  False        True

# Get summary of missingness 
airquality.isna().sum()

Date           0
Temperature    0
C02            366
dtype: int64
    
# Missingno (Useful package for visualizing and understanding missing data)
import missingno as msno
import matplotlib.pyplot as plt
# Visualize missingness
msno.matrix(airquality)
plt.show()

# Isolate missing and complete values aside
missing = airquality [airquality['C02'].isna()] 
complete = airquality [~airquality ['C02'].isna()]

# Describe complete DataFramee complete.describe()
      Temperature   C02
count 8991.000000   8991.000000
mean    18.317829      1.739584

# Describe missing DataFramee missing.describe()
      Temperature   CO2
count 366.000000    0.0
mean  -39.655738    NaN

# Missingness types
Missing Completely at Random (MCAR)
No systematic relationship between missing data and other values
Data entry errors when inputting data

Missing at Random (MAR)
Systematic relationship between missing data and other observed values
Missing ozone data for high temperatures

Missing Not at Random (MNAR)
Systematic relationship between missing data and unobserved values
Missing temperature values for high temperatures

# How to deal with missing data?
Simple approaches:
1. Drop missing data
2. Impute with statistical measures (mean, median, mode..)
More complex approaches:
1. Imputing using an algorithmic approach
2. Impute with machine learning models

# drop missing values
airquality_dropped = airquality.dropna(subset = ['CO2']) # drops all records that where  co2 is nan

# Replacing with statistical measures
co2_mean = airquality['CO2'].mean()
airquality_imputed = airquality.fillna({'CO2': co2_mean})

In [ ]:
# Record Linkage
# Minimum edit distance (Comparing the how similar is one string to another)
Least possible amount of steps needed to transition from one string to another #ex- REEDING to READING, Here the  minimum edit distance is 1. | ex2- INTENTION to EXECUTION, in this case the minimum edit distance is 5

+ Insertion
- Deletion
* Substitution
➡ Transposition

Minimum edit distance algorithms
Algorithm              Operations
Damerau-Levenshtein    insertion, substitution, deletion, transposition
Levenshtein            insertion, substitution, deletion
Hamming                substitution only
Jaro distance          transposition only
...
Possible packages: thefuzz
    
# simple string comparision
from thefuzz import fuzz
# compare reeding vs reading
fuzz.wratio('Reeding','Reading') # prints 86 where 0 will be less similar and 100 being the complete match

# Partial strings and different orderings
# Partial string comparision
fuzz.WRatio('Houston Rockets', 'Rockets') # prints 90

# Partial string comparision with different order
fuzz.WRatio('Houston Rockets vs Los Angeles Lakers', 'Lakers vs Rockets') # prints 86

# Comparisiion with arrays
# Import process
from thefuzz import process

# Define a string and array of possible matches
string = "Houston Rockets vs Los Angeles Lakers"
choices = pd.series(['Rockets vs Lakers', 'Lakers vs Rockets', 'Houston vs Los Angeles', 'Heat vs Bulls'])
process.extract(string, choices, limit=2) # prints [('Rockets vs Lakers', 86, 0), ('Lakers vs Rockets', 86, 1)]

# Collapsing categories with string similarity
Chapter 2
Use .replace() to collapse "eur" into "Europe"
What if there are too many variations?
"EU", "eur", "Europ", "Europa", "Erope", "Evropa" .

# Collapsing all of the state
# For each correct category
for state in categories['state']:
    #Find potential matches in states with typos
    matches = process.extract(state, survey['state'], limit = survey.shape[0])
    #for each potential match match
    for potential_match[1] >= 80:
        if potential_match[1] >= 80:
            #Replace typo with correct category
            survey.loc[survey['state'] == potential_match[0], 'state'] = state
            
            
# Record linkage
Data A
        Generate pairs -> Compare pairs -> Score pairs -> Link data
Data B

# Generating pairs
import recordLinkage

# Create indexing object
indexer = recordlinkage.Index()

# Generate pairs blocked on state
indexer.block('state')
pairs = indexer.index(census_A, census_B)
print(pairs) # prints MultiIndex(levels=[['rec-1007-org', 'rec-1016-org', 'rec-1054-org', 'rec-1066-org','rec-1070-org', 'rec-1075-org', 'rec-1080-org',15, 19, 57, 37, 70, 94]], names=['rec_id_1', 'rec_id_2'])

# Comparing the dataframes
# Generate the pairs
pairs = indexer.index(census_A, census_B)

# Create a compare object
compare_cl = recordlinkage.Compare()

# Find exact matches for pairs of date_of_birth and state
compare_cl.exact('date_of_birth', 'date_of_birth', label='date_of_birth')
compare_cl.exact('state', 'state', label='state')

# Find similar matches for pairs of surname and address_1 using string siimilarity
compare_cl.string('surname', 'surname', threshold=0.85, label='surname')
compare_cl.string('address_1'. 'address_1', threshold=0.85, label='address_1')

# Find matching pairs
potential_matches = compare_cl.compute(pairs, census_A, census_B)
print(potential_matches)
# prints below
rec_id_1        rec_id_2         date_of_birth  state surname address_1
rec-1070-org    rec-561-dup-0                0              1       0.0
                rec-2642-dup-0               0              1       0.0
                rec-608-dup-0                0              1       0.0
...
rec-1631-org    rec-4070-dup-0               0              1       0.0
                rec-4862-dup-0               0              1       0.0
                rec-629-dup-0                0              1       0.0

# Finding the only pairs we want
potential_matches[potential_matches.sum(axis=1) => 2]
# prints below
rec_id_1       rec_id_2         date_of_birth state surname address_1
rec-4878-org   rec-4878-dup-0               1     1     1.0       0.0
rec-417-org    rec-2867-dup-0               0     1     0.0       1.0
rec-3964-org   rec-394-dup-0                0     1     1.0       0.0
rec-1373-org   rec-4051-dup-0               0     1     1.0       0.0
               rec-802-dup-0                0     1     1.0       0.0

# Get the indices
matches.index # prints MultiIndex(levels=[['rec-1007-org', 'rec-1016-org', 'rec-1054-org', 'rec-1066-org','rec-1070-org', 'rec-1075-org', 'rec-1080-org', 'rec-110-org',

# Get he indices from census_b only
duplicate_rows = matches.index.get_level_values(1)
print(census_B_index) # prints Index(['rec-2404-dup-0', 'rec-4178-dup-0', 'rec-1054-dup-0', 'rec-4663-dup-0', 'rec-485-dup-0', 'rec-2950-dup-0', 'rec-1234-dup-0','rec-299-dup-0'])

# Finding duplicates in census_b
census_B_duplicates = census_B[census_B.index.isin(duplicate_rows)]

# Finding new rows in census_B
census_B_new = census_B[~census_B.index.isin(duplicate_rows)]

# Link the dataframe
full_census = census_A.append(census_B_new)